# Aprendizagem Automática Avançada
## Assigment 3
### Luís Ferreirinha Nº51127
### Christopher Anaya Nª60566

## Problem 2

We have created a class to implement our weighted average ensemble algorithm with fit and predict functions.  The data is loaded and then scaled.  We then implemented our algorithm with various base experts including Decision Trees and SVM Classifiers.  The SVMs return a better score, but we can combine the two sets of experts to get an even better ensemble.

In [9]:
# Imports
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

Let's start by creating a new class for this system.

In [3]:
def softmax(x):
    return np.exp(x) /  np.exp(x).sum()

class WeightedAverageEnsemble:
    """
    Objects of this class represent a Weighted Average Ensemble System
    These can instantiated with any type of experts in a list
    """
    experts = None
    weights = None
    X_train = None
    X_validation = None
    y_validation = None
    y_train = None

    def __init__(self, experts : list):
        self.experts = experts

    def fit(self, X_train, y_train):
        self.X_train, self.X_validation, self.y_train, self.y_validation = train_test_split(X_train, y_train, test_size=0.2)
        accuracies = []
        for model in self.experts:
            model.fit(self.X_train, self.y_train)
            model_preds = model.predict(self.X_validation)
            accuracies.append(accuracy_score(model_preds, self.y_validation))
        self.weights = softmax(accuracies)

    def predict(self, X_test):
        expert_predictions = []
        for model in self.experts:
            model_preds = model.predict(X_test)
            expert_predictions.append(model_preds)
        previsions = np.dot(self.weights, expert_predictions)
        return [round(prev) for prev in previsions]

Import the dataset and scale the data

In [4]:
# Loading the dataset
X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=212)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Now let's experiment with different types of classifiers as experts.
We will begin by utilizing two Decision Trees with gini and entropy as criterions.

In [5]:
system_1 = WeightedAverageEnsemble([DecisionTreeClassifier(criterion="entropy"), DecisionTreeClassifier(criterion="gini")])
system_1.fit(X_train_scaled, y_train)
preds_1 = system_1.predict(X_test_scaled)
print("Accuracy:")
print(accuracy_score(preds_1, y_test))

Accuracy:
0.9122807017543859


Two SVM's with polynomial and RBF kernel

In [6]:
system_2 = WeightedAverageEnsemble([SVC(kernel='rbf'), SVC(kernel='poly')])
system_2.fit(X_train_scaled, y_train)
preds_2 = system_2.predict(X_test_scaled)
print("Accuracy:")
print(accuracy_score(preds_2, y_test))

Accuracy:
0.9824561403508771


A mixture of the previous two cases

In [7]:
system_3 = WeightedAverageEnsemble([DecisionTreeClassifier(criterion="entropy"), \
                DecisionTreeClassifier(criterion="gini"), SVC(kernel='rbf'), SVC(kernel='poly')])
system_3.fit(X_train_scaled, y_train)
preds_3 = system_3.predict(X_test_scaled)
print("Accuracy")
print(accuracy_score(preds_3, y_test))

Accuracy
0.9473684210526315


And finally we will experiment with a KNN classifier and a Gaussian Naive Bayes classifier

In [11]:
system_4 = WeightedAverageEnsemble([KNeighborsClassifier(n_neighbors=4), GaussianNB()])
system_4.fit(X_train_scaled, y_train)
preds_4 = system_4.predict(X_test_scaled)
print("Accuracy:")
print(accuracy_score(preds_4, y_test))

Accuracy:
0.956140350877193


In conclusion, our work showed that a classifier that uses a weighted average ensemble approach can effectively combine the predictions of multiple experts to achieve higher accuracy in classification tasks. We found that the use of two simple experts can provide a good level of accuracy (e.g. Decision Trees and KNN + Naive Bayes). The usage of these simpler experts will correspond to a lower train and prediction time for larger datasets. However, we also observed that more complex experts can lead to even better accuracy, but of course this will probably lead to higher train times for larger datasets.